## We will mostly focus on the following
 - Mendeleev (https://pypi.org/project/mendeleev/)
 - Pymatgen (https://pymatgen.org/)
 - Atomic Simulation Environment (ASE) (https://wiki.fysik.dtu.dk/ase/)
 - RDkit (https://www.rdkit.org/)

## Mendeleev
Python based package to extract different properties of elements in the periodic table (118 atoms, as of July, 2022).

#### Extracting elemental properties

Requirements:
We need to install the following packages for accessing the tutorial materials
pip install mendeleev[vis]
pip install ase
pip install pymatgen

In [ ]:
#!pip install mendeleev[vis]
#!pip install bokeh
#!pip install ase
#!pip install pymatgen

In [ ]:
from mendeleev import element

We could eg. look up for an element 'Mg'. [Feel free to use any elements you like here]

In [ ]:
ele=element('Mg')

In [ ]:
print(dir(ele))

Lets take a look what is stored inside 'ele' !!

In [ ]:
ele

We see a lot of values stored. Lets see what are values available for dipole polarizability, density, lattice_constant'
electronic configuration (ec), ionic_radii, Paulings electronegativity etc !!!

In [ ]:
ele.dipole_polarizability

In [ ]:
ele.lattice_constant

In [ ]:
ele.density

In [ ]:
ele.ec

In [ ]:
print(ele.ionic_radii)

In [ ]:
ele.ionic_radii[0].ionic_radius 


In [ ]:
ele.ionic_radii[10].ionic_radius 

Did we get error message while executing above code? Could we fix it if so? Yes !!! 

In [ ]:
ele.en_pauling # Pauling's electronegativity


There is another way of getting all the properties for any elements in the periodic table. Calling directly by the 
element name as below

In [ ]:
from mendeleev import Mg

For example for getting density of Mg we can use. 

In [ ]:
Mg.density

#### Extracting as pandas dataframe

In [ ]:
from mendeleev.fetch import fetch_table
all_data = fetch_table('elements')
all_data.head()
#all_data.info()
#all_data.dtypes
#all_data.shape

In [ ]:
all_data.shape

In [ ]:
all_data.info()

In [ ]:
all_data.head()

#### Visualization

In [ ]:
from mendeleev.vis import periodic_table
periodic_table()
fig=periodic_table(attribute='lattice_constant', title="Lattice constants")
#fig.show()
fig.show()

In [ ]:
fig=periodic_table(attribute='en_pauling', title="Paulings Electronegativity")

In [ ]:
fig.show()

In [ ]:
#fig = periodic_table_bokeh(elements, attribute="atomic_radius", colorby="attribute")
#show(fig)

In [ ]:
#Using bokeh
from mendeleev.vis import create_vis_dataframe, periodic_table_plotly
elements = create_vis_dataframe()
periodic_table_plotly(elements)
from bokeh.plotting import show, output_notebook
from mendeleev.vis import periodic_table_bokeh
output_notebook()
fig = periodic_table_bokeh(elements)
show(fig)

In [ ]:
#show(fig)
fig=periodic_table_bokeh(elements,attribute='lattice_constant', title="Lattice Constants")
show(fig)

## Atomic Simulation Environment (ASE)
Python based environment to prepare, perform, and analyze the atomistic simulation. It is one of the most popular tools in computational physics and chemistry.

### Preparing Atomistic Simulations
 - Structure (generation + visualization)
 - Determine parameters to run the calculations ( which optimizer, which functional, what energy and force cutoff, etc.)

In [ ]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate
from ase.visualize import view

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())

#### Using Atoms object in ASE to build molecules

In [ ]:
from ase import Atoms
d = 1.18
co2 = Atoms('CO2', positions=[(0, 0, 0),(0,0,-d), (0, 0, d)]) # cell=[float,float,float] #pbe=[int,int,int]
co2

#### Visualizing the molecule

In [ ]:
#view(co, viewer='x3d')
#view(co, viewer='ngl')

view(co2)

#### Writting the data to a structure file (https://wiki.fysik.dtu.dk/ase/ase/io/io.html)

In [ ]:
co2.write('co2.xyz') # If not periodic boundary condition is set .xyz and .cif are the format for saving image (try .vasp!)

Repeating above steps for periodic boundary condition.

In [ ]:
co2 = Atoms('CO2', positions=[(0, 0, 0), (0,0,-d), (0, 0, d)],cell=[10,10,10], pbc=(2,0,0))
view(co2)
fig1=view(co2)


ASE allows to write the coordinates of the atoms in the above supercell to a datafile


In [ ]:
co2.write('co2.xyz') # Using periodic boundary condition; different file types are allowed
co2.write('co2.cif')

In [ ]:
print(dir(co2))

In [ ]:
print(co2.get_chemical_symbols())
print(co2.get_center_of_mass())
print(co2.get_positions())
print(co2.get_volume())
print(co2.get_cell())

#### Feel free to play around with other attributes above

Although we built a molecule on our own ussing the information of bond lengths. ASE has in built library that contains
a lot of molecules. Following is the list of structures that are available in ASE

In [ ]:
from ase.collections import g2
ase_avail_str=g2.names
print(ase_avail_str)

Below we can quickly check if the given molecule is available in the g2 library. eg. Lets try if benzene ie. 'C6H6' is available there

In [ ]:
'C6H6' in ase_avail_str

We see benzene exists already in the g2 library. Therefore we don't need to construct it again (as we did a while ago for CO2.)
[Note: the molecule 'CO2' also exists here]. Could we visualize Benzene then? Could we add this molecule in a box of dimension 
20x20x20 ? Could we also save the coordinates of each atoms in the standard file type (eg .xyz, .vasp, .cif, etc.)? 

Hint: The answer to all of the above questions is yes!!! . We could simply follow the procedure we adopted above for CO2

In [ ]:
from ase.build import molecule
benz=molecule('C6H6')
#print(dir(benz))
benz.set_cell([20,20,20])
view(benz)
print(benz.get_chemical_symbols())
print(benz.get_center_of_mass())
print(benz.get_positions())
print(benz.get_volume())
print(benz.get_cell())


Could we place benzene at the center of the supercell? Yes!!

In [ ]:
benz.set_center_of_mass([-10,-10,-10])
view(benz)

#### Building solids (bulk)
Silver has a fcc crystal lattice structure. Imagine a cube with atoms at each corners and at the center of each face.
We can get its lattice constant from mendeleev and use it to build the bulk.

In [ ]:
from mendeleev import Ag
lat_ag=Ag.lattice_constant
from ase.build import bulk
ag_prim=bulk('Ag','fcc',lat_ag)
ag_ortho=bulk('Ag','fcc',lat_ag,orthorhombic=True)
ag_conv=bulk('Ag','fcc',lat_ag,cubic=True)
#view(ag_prim)
#view(ag_ortho)
#view(ag_conv)
#ag_prim.get_cell()
#ag_ortho.get_cell()
#ag_conv.get_cell()
ag_prim.get_positions()
#ag_ortho.get_positions()
#ag_conv.get_positions()
ag_prim.write('Ag.cif')

In [ ]:
from mendeleev import Na
lat_na=Na.lattice_constant
from ase.build import bulk
na_prim=bulk('Na','bcc',lat_na)
na_conv=bulk('Na','bcc',lat_na,cubic=True)
#view(na_prim)

view(na_conv)
#na_prim.get_cell()

#na_conv.get_cell()
#na_prim.get_positions()
na_conv.get_positions()

#### Building surfaces

In [ ]:
from ase.build import fcc111

In [ ]:
surf_ag = fcc111('Ag',a=lat_ag, size=(4,4,5))
#slab = fcc111('Al', size=(2,2,3))
add_adsorbate(surf_ag, 'H', 1.5, 'ontop')
surf_ag.center(vacuum=10.0, axis=2)
view(surf_ag)
#surf_ag.get_cell()

In [ ]:
len(surf_ag.get_positions())